In [33]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge, Lasso

## 1- Modèle GLM

In [18]:
X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')

X['maille_safran'] = X['maille_safran'].astype(str)
Y['Mailles'] = Y['Mailles'].astype(str)
Y.rename(columns={'Mailles': 'maille_safran', 'Année': 'Année'}, inplace=True)

X['maille_safran'] = X['maille_safran'].astype(int)
Y['maille_safran'] = Y['maille_safran'].astype(float).astype(int)

data = pd.merge(X, Y, on=['maille_safran', 'Année'], how='inner')
data['IFM_moy'] = data.iloc[:, 2:32].mean(axis=1)  #On regresse sur ça mais je capte pas bien ce que c'est


In [19]:
# Modèle GLM avec lien logit
model = smf.glm(formula='Q("Feu en été") ~ IFM_moy', data=data, family=sm.families.Binomial())
result = model.fit()

print(result.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:        Q("Feu en été")   No. Observations:                 2448
Model:                            GLM   Df Residuals:                     2446
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                   -inf
Date:                Thu, 18 Apr 2024   Deviance:                       4991.0
Time:                        16:28:05   Pearson chi2:                 2.68e+03
No. Iterations:                     5   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5263      0.087      6.030      0.0

/opt/mamba/lib/python3.11/site-packages/statsmodels/genmod/generalized_linear_model.py:1891: RuntimeWarning: invalid value encountered in scalar subtract
  prsq = 1 - np.exp((self.llnull - self.llf) * (2 / self.nobs))


# 2 - Train/Test

## a. Regression Logistique

# les différents modèles qu'on pourrait essayer de tester??

SVM (Support Vector Machine) : Les SVM sont également populaires pour la classification binaire. Ils sont efficaces dans les espaces de grande dimension et peuvent bien fonctionner avec des données numériques.

Forêts aléatoires (Random Forests) : Les forêts aléatoires sont une extension des arbres de décision et sont moins sensibles au surajustement. Elles sont robustes et peuvent gérer de grandes quantités de données.

Réseaux de neurones artificiels (Neural Networks) : Les réseaux de neurones peuvent être très puissants pour la classification binaire, mais ils peuvent nécessiter plus de données et de ressources de calcul.

In [28]:
# On merge les deux tables
X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')

X['maille_safran'] = X['maille_safran'].astype(str)
Y['Mailles'] = Y['Mailles'].astype(str)
Y.rename(columns={'Mailles': 'maille_safran', 'Année': 'Année'}, inplace=True)

X['maille_safran'] = X['maille_safran'].astype(int)
Y['maille_safran'] = Y['maille_safran'].astype(float).astype(int)

data = pd.merge(X, Y, on=['maille_safran', 'Année'], how='inner')

In [29]:
data

,maille_safran,01 Jun,02 Jun,03 Jun,04 Jun,05 Jun,06 Jun,07 Jun,08 Jun,09 Jun,...,23 Aug,24 Aug,25 Aug,26 Aug,27 Aug,28 Aug,29 Aug,30 Aug,Année,Feu en été
0,7741,6.6,4.4,4.5,0.1,0.1,0.0,0.5,0.7,1.5,...,31.9,25.9,24.8,24.5,18.6,24.6,31.2,28.4,2006,1
1,7742,8.7,4.8,6.1,0.0,0.0,0.0,0.4,0.8,1.9,...,34.3,24.8,24.3,26.2,19.5,25.2,34.5,30.5,2006,1
2,7743,8.0,2.5,2.9,0.0,0.0,0.0,0.3,0.5,0.9,...,31.9,21.7,22.0,28.1,27.7,23.7,29.8,30.3,2006,1
3,7823,8.3,10.2,9.3,0.0,0.0,0.0,0.5,0.7,2.0,...,33.3,24.5,27.2,25.6,19.4,26.7,35.8,31.5,2006,1
4,7824,7.8,6.5,7.4,0.0,0.0,0.0,0.4,0.7,1.8,...,35.1,21.0,23.3,26.7,20.6,26.7,36.4,31.5,2006,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,8814,5.2,0.3,3.0,0.9,0.9,0.9,6.2,6.8,13.9,...,15.8,17.8,15.7,20.4,16.3,0.7,0.1,0.6,2022,0
2444,8815,3.6,0.3,2.1,0.5,0.4,0.2,2.5,3.2,10.7,...,14.1,15.5,13.9,17.0,12.0,0.4,0.1,0.7,2022,0
2445,8893,5.4,5.0,13.0,6.8,8.4,7.8,11.8,11.4,26.4,...,18.7,16.8,11.8,18.1,11.7,3.2,4.4,9.9,2022,0
2446,8894,6.3,5.2,12.1,7.6,9.6,7.6,14.0,15.3,26.1,...,17.9,16.4,17.2,19.6,12.4,3.5,3.5,6.8,2022,0


In [13]:
# Supposons que X et y représentent tes données et ta variable cible
# X est un tableau numpy des caractéristiques
# y est un tableau numpy de la variable cible binaire
# Sélectionner les colonnes pour les fonctionnalités et la variable cible
Xt = data.drop(columns=['Feu en été'])  # Exclure la colonne 'résultat' de X
y = data['Feu en été']

In [14]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2, random_state=42)

# Initialiser le modèle de régression logistique
model = LogisticRegression()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédire les étiquettes sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer les performances du modèle en utilisant l'exactitude (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6959183673469388


/opt/mamba/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy  0,7 environ

## b. Arbre de décision

In [23]:
Xt = data.drop(columns=['Feu en été'])  # Exclure la colonne 'résultat' de X
y = data['Feu en été']

In [16]:
# Diviser les données en ensembles d'entraînement et de test (si ce n'est pas déjà fait)
X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2, random_state=42)

# Initialiser le modèle d'arbre de décision
model = DecisionTreeClassifier()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédire les étiquettes sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer les performances du modèle en utilisant l'exactitude (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.763265306122449


Accuracy  0,76 environ

Essayer d'améliorer l'accuracy de l'arbre de décision en jouant sur les hyperparamètres

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Définir les hyperparamètres à rechercher
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialiser le modèle d'arbre de décision
model = DecisionTreeClassifier()

# Initialiser la recherche d'hyperparamètres
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Effectuer la recherche d'hyperparamètres sur l'ensemble d'entraînement
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres trouvés
print("Meilleurs paramètres:", grid_search.best_params_)

# Obtenir la meilleure performance trouvée lors de la validation croisée
print("Meilleure précision (score CV):", grid_search.best_score_)

# Utiliser le modèle avec les meilleurs paramètres pour prédire sur l'ensemble de test
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Évaluer les performances du modèle sur l'ensemble de test
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy sur l'ensemble de test:", accuracy)

/opt/mamba/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Meilleurs paramètres: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Meilleure précision (score CV): 0.7574155749256224
Accuracy sur l'ensemble de test: 0.7673469387755102


Améloration minime et pas terrible, mais probablement car 5 cross validation, trop importante par rapport aux données que l'on a

## c. Random Forests

In [23]:
Xt = data.drop(columns=['Feu en été'])  # Exclure la colonne 'résultat' de X
y = data['Feu en été']

In [20]:
# Initialiser le modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy sur l'ensemble de test:", accuracy)

Accuracy sur l'ensemble de test: 0.8061224489795918


Accuracy 0,8 : de mieux en mieux !

## d. Ridge et Lasso

In [34]:
Xt = data.drop(columns=['Feu en été'])  # Exclure la colonne 'résultat' de X
y = data['Feu en été']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2, random_state=42)

# Initialiser le modèle Ridge
ridge_model = Ridge()

# Définir les hyperparamètres à rechercher
param_grid_ridge = {'alpha': [0.1, 1, 10]}

# Initialiser la recherche d'hyperparamètres
grid_search_ridge = GridSearchCV(ridge_model, param_grid_ridge, cv=5)

# Ajuster le modèle Ridge aux données
grid_search_ridge.fit(X_train, y_train)

# Obtenir le meilleur modèle Ridge
best_ridge_model = grid_search_ridge.best_estimator_

# Initialiser le modèle Lasso
lasso_model = Lasso()

# Définir les hyperparamètres à rechercher
param_grid_lasso = {'alpha': [0.1, 1, 10]}

# Initialiser la recherche d'hyperparamètres
grid_search_lasso = GridSearchCV(lasso_model, param_grid_lasso, cv=5)

# Ajuster le modèle Lasso aux données
grid_search_lasso.fit(X_train, y_train)

# Obtenir le meilleur modèle Lasso
best_lasso_model = grid_search_lasso.best_estimator_

print("Meilleur score Ridge :", grid_search_ridge.best_score_)
print("Meilleurs paramètres Ridge :", grid_search_ridge.best_params_)
print("Meilleur score Lasso :", grid_search_lasso.best_score_)
print("Meilleurs paramètres Lasso :", grid_search_lasso.best_params_)

Meilleur score Ridge : 0.22733317859093932
Meilleurs paramètres Ridge : {'alpha': 10}
Meilleur score Lasso : 0.18952053930699891
Meilleurs paramètres Lasso : {'alpha': 0.1}


# 3 - GLM

In [30]:
Xt = data.drop(columns=['Feu en été', 'maille_safran', 'Année'])  # Exclure la colonne 'résultat' de X
y = data['Feu en été']

In [32]:
# Ajouter une constante à la matrice X pour l'intercept
Xt = sm.add_constant(Xt)

# Initialiser le modèle GLM (par exemple, une régression logistique)
model = sm.GLM(y, Xt, family=sm.families.Binomial())

# Ajuster le modèle aux données
result = model.fit()

# Afficher un résumé des résultats
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             Feu en été   No. Observations:                 2448
Model:                            GLM   Df Residuals:                     2356
Model Family:                Binomial   Df Model:                           91
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                   -inf
Date:                Tue, 23 Apr 2024   Deviance:                       4194.1
Time:                        15:17:03   Pearson chi2:                 3.28e+03
No. Iterations:                     7   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2986      0.492     -2.639      0.0

/opt/mamba/lib/python3.11/site-packages/statsmodels/genmod/generalized_linear_model.py:1891: RuntimeWarning: invalid value encountered in scalar subtract
  prsq = 1 - np.exp((self.llnull - self.llf) * (2 / self.nobs))
